In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

# =======================
# 1. CARGA Y FEATURE ENGINEERING
# =======================
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
test_ids = test["PassengerId"]

full_data = pd.concat([train, test], sort=False)

# Title simplificado
full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
full_data["Title"] = full_data["Title"].replace(['Mlle', 'Ms'], 'Miss')
full_data["Title"] = full_data["Title"].replace(['Mme'], 'Mrs')
full_data["Title"] = full_data["Title"].replace(
    ['Countess', 'Lady', 'Sir', 'Jonkheer', 'Don', 'Dona', 'Capt', 'Col', 'Major', 'Dr', 'Rev'],
    'Rare')

# Tamaño familiar
full_data["FamilySize"] = full_data["SibSp"] + full_data["Parch"] + 1
full_data["IsAlone"] = (full_data["FamilySize"] == 1).astype(int)

# Cabina
full_data["CabinInitial"] = full_data["Cabin"].str[0].fillna('U')
full_data["HasCabin"] = full_data["Cabin"].notnull().astype(int)

# Interacciones y derivados
full_data["FarePerPerson"] = full_data["Fare"] / full_data["FamilySize"]
full_data["Age*Class"] = full_data["Age"] * full_data["Pclass"]
full_data["TicketPrefix"] = full_data["Ticket"].str.extract(r'([A-Za-z]+)', expand=False).fillna("NONE")

# Separar nuevamente
train = full_data[~full_data["Survived"].isna()]
test = full_data[full_data["Survived"].isna()].drop(columns=["Survived"])
X = train.drop(columns=["Survived", "Name", "Ticket", "PassengerId", "Cabin"])
y = train["Survived"]
X_test_final = test.drop(columns=["Name", "Ticket", "PassengerId", "Cabin"])

# =======================
# 2. PIPELINE DE PREPROCESAMIENTO
# =======================
cat_cols = ["Sex", "Embarked", "Title", "CabinInitial", "Pclass", "TicketPrefix"]
num_cols = ["Age", "Fare", "FamilySize", "IsAlone", "FarePerPerson", "Age*Class", "HasCabin"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# =======================
# 3. MODELOS BASE Y STACKING
# =======================
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=2,
    random_state=42
)

lgbm = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=5,
    num_leaves=25,
    min_child_samples=20,
    random_state=42
)

lr = LogisticRegression(max_iter=1000)

stacked_model = StackingClassifier(
    estimators=[("rf", rf), ("lgbm", lgbm)],
    final_estimator=lr,
    passthrough=True,
    cv=5,
    n_jobs=-1
)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", stacked_model)
])

# =======================
# 4. VALIDACIÓN CRUZADA ROBUSTA
# =======================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy")
print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

# =======================
# 5. ENTRENAMIENTO FINAL Y PREDICCIÓN
# =======================
clf.fit(X, y)
final_preds = clf.predict(X_test_final)

submission = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": final_preds.astype(int)
})
submission.to_csv("../data/submission_stacking.csv", index=False)

In [5]:
print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

CV Accuracy: 0.8271 ± 0.0145


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

# =======================
# 1. CARGA Y FEATURE ENGINEERING
# =======================
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
test_ids = test["PassengerId"]

full_data = pd.concat([train, test], sort=False)

# Título simplificado
full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)
full_data["Title"] = full_data["Title"].replace(['Mlle', 'Ms'], 'Miss')
full_data["Title"] = full_data["Title"].replace(['Mme'], 'Mrs')
full_data["Title"] = full_data["Title"].replace(
    ['Countess', 'Lady', 'Sir', 'Jonkheer', 'Don', 'Dona', 'Capt', 'Col', 'Major', 'Dr', 'Rev'],
    'Rare')
title_map = {"Mr": "Mr", "Mrs": "Mrs", "Miss": "Miss", "Master": "Master"}
full_data["Title"] = full_data["Title"].map(lambda x: title_map.get(x, "Rare"))

# Family size
full_data["FamilySize"] = full_data["SibSp"] + full_data["Parch"] + 1
full_data["IsAlone"] = (full_data["FamilySize"] == 1).astype(int)

# Fare por persona
full_data["FarePerPerson"] = full_data["Fare"] / full_data["FamilySize"].replace(0, 1)
full_data["FarePerPerson"] = full_data["FarePerPerson"].fillna(full_data["FarePerPerson"].median())

# Interacción edad y clase
full_data["Age*Class"] = full_data["Age"] * full_data["Pclass"]

# Tiene cabina
full_data["HasCabin"] = full_data["Cabin"].notnull().astype(int)

# Separar train y test
train = full_data[~full_data["Survived"].isna()]
test = full_data[full_data["Survived"].isna()].drop(columns=["Survived"])
X = train.drop(columns=["Survived", "Name", "Ticket", "PassengerId", "Cabin"])
y = train["Survived"]
X_test_final = test.drop(columns=["Name", "Ticket", "PassengerId", "Cabin"])

# =======================
# 2. PIPELINE DE PREPROCESAMIENTO
# =======================
cat_cols = ["Sex", "Embarked", "Title", "Pclass"]
num_cols = ["Age", "Fare", "FamilySize", "IsAlone", "FarePerPerson", "Age*Class", "HasCabin"]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# =======================
# 3. STACKING MODEL
# =======================
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=2,
    random_state=42
)

lgbm = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=5,
    num_leaves=25,
    min_child_samples=20,
    random_state=42
)

lr = LogisticRegression(max_iter=1000)

stacked_model = StackingClassifier(
    estimators=[("rf", rf), ("lgbm", lgbm)],
    final_estimator=lr,
    passthrough=True,
    cv=5,
    n_jobs=-1
)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", stacked_model)
])

# =======================
# 4. VALIDACIÓN CRUZADA ROBUSTA
# =======================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy")
print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")

# =======================
# 5. ENTRENAMIENTO FINAL Y PREDICCIÓN
# =======================
clf.fit(X, y)
final_preds = clf.predict(X_test_final)

submission = pd.DataFrame({
    "PassengerId": test_ids,
    "Survived": final_preds.astype(int)
})
submission.to_csv("../data/submission_stacking_cleaned.csv", index=False)

<>:23: SyntaxWarning: invalid escape sequence '\.'
<>:23: SyntaxWarning: invalid escape sequence '\.'
/var/folders/15/59msbdnd3179pytbr6876h9m0000gn/T/ipykernel_3582/3963837698.py:23: SyntaxWarning: invalid escape sequence '\.'
  full_data["Title"] = full_data["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)


[LightGBM] [Info] Number of positive: 273, number of negative: 439
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000542 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 424
[LightGBM] [Info] Number of data points in the train set: 712, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383427 -> initscore=-0.475028
[LightGBM] [Info] Start training from score -0.475028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastian

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 274, number of negative: 439
[LightGBM] [Info] Auto-choosing r

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastian

...

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastian

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/sebastianfelipeurzuaborquez/Proyectos/Titanic Dataset/.venv/lib/python3.13/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/sebastianfelipeurzuaborquez/Proyectos/T

In [ ]:
print(f"CV Accuracy: {scores.mean():.4f} ± {scores.std():.4f}")
#fue peor

CV Accuracy: 0.8339 ± 0.0114
